XB1: freq: [400, 40, 4] Hz

XB2: doserate: [3.2, ~~0.32~~] MGy/s

**XB3: t_pulse: [50, 500] microseconds**

In [ ]:
TOKEN = 'XB3'

In [ ]:
import json
import os
from collections import defaultdict
from IPython.display import HTML
import numpy as np
import matplotlib
from matplotlib import animation
import matplotlib.pyplot as plt

from analysis import (
    parse_results, get_token_groups, plot_token_groups, animate, average_length_by_weight,
    tokens, counter_tokens, total_of, sum_up, mk_cumulative_interpolator
)
%matplotlib inline

In [ ]:
def read(path):
    tout, Cout, info = parse_results(path)
    tokg, ctrg, spg = get_token_groups(Cout)
    init = []
    for sk in spg['polymeric']:
        if Cout[sk][0] > 0:
            init.append((sk, Cout[sk][0]))
    (sk0, ic0), = init
    return locals()

In [ ]:
cases = __import__('cases_%s' % TOKEN)

In [ ]:
cases.int_indices.index((0,0,0))

In [ ]:
def res(*, imf, iatmo, itpulse):
    if itpulse == 0:
        casesXB1 = __import__('cases_XB1')
        ifreq = 1
        return 'XB1-res/XB1_0_%d.txt' % casesXB1.int_indices.index((imf, iatmo, ifreq))
    else:
        return 'XB3-res/XB3_0_%d.txt' % cases.int_indices.index((imf, iatmo, itpulse - 1))
#tps = range(len(cases.tpulses))
tps = [0, 1, 2]

def _varied(tpulse_idx):
    if tpulse_idx == 0:
        return 'varied-XB1-1.json'
    else:
        return 'varied-%s-%d.json' % (TOKEN, tpulse_idx-1)

interpols = []
for tpulse_idx in tps:
    dur, varied = json.load(open(_varied(tpulse_idx)))
    dr = varied.pop('doserate')
    assert not varied
    interpols.append(mk_cumulative_interpolator(dur, dr))

In [ ]:
all_air, all_nit = {}, {}
for imf in range(len(cases.mass_fracs)):
    for iatmo in range(2):
        for itpulse in tps:
            if cases.atmos[iatmo]['N2O'] == 0 and cases.atmos[iatmo]['O2'] > 0.2e-3:
                r = all_air
            elif cases.atmos[iatmo]['N2O'] > 0.020 and cases.atmos[iatmo]['O2'] == 0:
                r = all_nit
            else:
                raise ValueError("Could not determine atmosphere type")

            r[imf, itpulse] = read(res(imf=imf, iatmo=iatmo, itpulse=itpulse))

In [ ]:
assert np.isclose(r[0,0]['tout'][-1], interpols[0][0][-1])

In [ ]:
interpols[0][2](r[0,0]['tout'][-1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 6))
xi = tps[2]
for i, k in enumerate(all_air[0, 0]['spg']['ord_rad']):
    ax.semilogy(all_air[0, xi]['tout'], all_air[0, xi]['Cout'][k], ls=['-',':','-.'][i % 3], label=k)
ax.set_ylim([1e-15, 100])
ax.legend()

In [ ]:
#plot_token_groups(tout, Cout, tokg)

In [ ]:
#anim1 = animate(tout, Cout, tokg)
#HTML(anim1.to_html5_video())

In [ ]:
def plot_conc(series_air, series_nit, ip1d, sks='H2O2 O2 N2O HO2'.split()):
    fig, all_axes = plt.subplots(len(series_air), 1 + len(sks), figsize=(16, 3*len(series_air)), sharey='col')
    assert len(series_air) == len(series_nit)
    for axes, air, nit in zip(all_axes, series_air, series_nit):
        for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
            axes[0].axhline(int(cont['sk0'][1:].split('a')[0]), ls='--', lw=0.5, alpha=0.5, c='k')
            axes[0].plot(ip1d(cont['tout'])/1e3, average_length_by_weight(cont['Cout'], cont['tokg']), label=lbl)
            for i in range(len(sks)):
                axes[i+1].plot(ip1d(cont['tout'])/1e3, cont['Cout'][sks[i]], label=lbl, alpha=0.7)
                axes[i+1].set_title(sks[i])
                axes[i+1].set_yscale('log')
        for ax in axes:
            ax.set_xlabel('Dose / kGy')
        axes[0].legend()
        axes[1].legend()
        _ = axes[0].set_title(r'Average length by weight')

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
if TOKEN == 'W2':
    for scal in range(5):
        plot_conc([all_air[scal, 0], all_air[scal, 1]], [all_nit[scal, 0], all_nit[scal, 1]])
elif TOKEN == 'W3':
    plot_conc(*[[cont[0, mf] for mf in mfs] for cont in [all_air, all_nit]])
elif TOKEN == 'XB3':
    for tpulse in tps:
        cx, cy, ip1d = interpols[tpulse]
        plot_conc(*[[cont[mf, tpulse] for mf in range(len(cases.mass_fracs))] for cont in [all_air, all_nit]], ip1d)
        plt.suptitle("t_pulse = %.1e s" % (5e-6 if tpulse == 0 else cases.tpulses_s[tpulse-1]))
        plt.tight_layout()
        plt.subplots_adjust(left=0.1, wspace=0.9, top=0.9)

In [ ]:
def plot_tokens(series_air, series_nit, ip1d):
    assert len(series_air) == len(series_nit)
    fig, all_axes = plt.subplots(len(series_air), len(tokens[1:]), figsize=(16, 3*len(series_air)), sharey='col')
    for axes, air, nit in zip(all_axes, series_air, series_nit):
        for tk, ax in zip(tokens[1:], axes):
            for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
                ax.plot(ip1d(cont['tout']), total_of(cont['Cout'], cont['tokg'][tk])/cont['ic0'], label=lbl)
            ax.legend()
            ax.set_title(tk)
            ax.set_xlabel('Dose / Gy')
            ax.set_yscale('log')
    

In [ ]:
len(all_air[0, 0]['Cout']['H'])

In [ ]:
if TOKEN == 'W2':
    for scal in range(5):
        plot_tokens([all_air[scal, 0], all_air[scal, 1]], [all_nit[scal, 0], all_nit[scal, 1]])
elif TOKEN == 'W3':
    plot_tokens(*[[cont[0, mf] for mf in mfs] for cont in [all_air, all_nit]])
elif TOKEN == 'XB3':
    for tpulse in tps:
        cx, cy, ip1d = interpols[tpulse]
        plot_tokens(*[[cont[mf, tpulse] for mf in range(len(cases.mass_fracs))] for cont in [all_air, all_nit]], ip1d)
        plt.suptitle("t_pulse = %.1e s" % (5e-6 if tpulse == 0 else cases.tpulses_s[tpulse-1]))
        plt.tight_layout()
        plt.subplots_adjust(left=0.1, wspace=0.9, top=0.9)    

In [ ]:
def plot_counters(series_air, series_nit, ip1d):
    assert len(series_air) == len(series_nit)
    fig, all_axes = plt.subplots(len(series_air), len(counter_tokens), figsize=(16, 3*len(series_air)), sharey='col')
    for axes, air, nit in zip(all_axes, series_air, series_nit):
        for ct, ax in zip(counter_tokens, axes):
            for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
                ax.plot(ip1d(cont['tout']), sum_up(cont['Cout'], cont['ctrg'][ct])/cont['ic0'], label=lbl)
            ax.set_xlabel('Dose / Gy')
            ax.legend()
            ax.set_title(ct)
            #ax.set_ylim([200, 300.]) ###############!!!!!!!!!!!!

In [ ]:
if TOKEN == 'W2':
    for scal in range(5):
        plot_counters([all_air[scal, 0], all_air[scal, 1]], [all_nit[scal, 0], all_nit[scal, 1]])
elif TOKEN == 'W3':
    plot_counters(*[[cont[0, mf] for mf in mfs] for cont in [all_air, all_nit]])
elif TOKEN == 'XB3':
    for tpulse in tps:
        cx, cy, ip1d = interpols[tpulse]
        plot_counters(*[[cont[mf, tpulse] for mf in range(len(cases.mass_fracs))] for cont in [all_air, all_nit]], ip1d)
        plt.suptitle("t_pulse = %.1e s" % (5e-6 if tpulse == 0 else cases.tpulses_s[tpulse-1]))
        plt.tight_layout()
        plt.subplots_adjust(left=0.1, wspace=0.9, top=0.9)

In [ ]:
# This cell generates figures for the manuscript
exec(open('ms.inc.py').read())